#Step 1: Caricamento del Dataset
Questo step prepara il dataset CNN/DailyMail utilizzando la libreria Hugging Face datasets. Ti guiderò a:

- Installare le librerie necessarie.
- Caricare e visualizzare un esempio di articolo e riassunto.

In [4]:
# Assicurati di avere le librerie installate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [9]:
from datasets import load_dataset

# Caricamento del dataset CNN/DailyMail
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Suddivisione dei dati
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

# Visualizza un esempio
print(f"Esempio articolo: {train_data[0]['article']}\n")
print(f"Esempio riassunto: {train_data[0]['highlights']}")


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Esempio articolo: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office cha

#Step 2: Preprocessing del Testo.
In questo passaggio, prepareremo i dati del dataset per essere utilizzati dal modello di abstractive summarization. Questo include la tokenizzazione e la gestione delle lunghezze variabili.
##Obiettivi dello Step 2
- Tokenizzare articoli e riassunti usando il tokenizer del modello T5.
- Truncare o applicare padding per uniformare la lunghezza dei token.
- Preparare un preprocessore per automatizzare questi passaggi.


In [10]:
# Assicurati di avere la libreria transformers installata
!pip install transformers

In [16]:
from transformers import AutoTokenizer

# Inizializza il tokenizer per T5
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Funzione di preprocessing
def preprocess_function(examples):
    # Prendi l'articolo come input e il riassunto come output
    inputs = examples['article']
    targets = examples['highlights']

    # Tokenizza l'articolo (input) e il riassunto (output)
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Tokenizza il riassunto e aggiungi come "labels" per il training
    labels = tokenizer(
        targets,
        max_length=150,
        truncation=True,
        padding="max_length"
    )

    # Aggiungi le labels al dizionario degli input
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocessing dei dati
processed_train_data = train_data.map(preprocess_function, batched=True)
processed_validation_data = validation_data.map(preprocess_function, batched=True)

# Mostra un esempio preprocessato
print(processed_train_data[0])


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

##Spiegazione del Codice
- max_length e padding: Limitiamo la lunghezza massima del testo a 512 token per l'articolo e 150 per il riassunto. Applichiamo padding per uniformare le dimensioni.
- labels: Sono le tokenizzazioni dei riassunti e verranno utilizzate come target per il modello.

##Risultati Attesi
Il dataset preprocessato mostrerà un dizionario con:

- input_ids: I token dell'articolo.
- labels: I token del riassunto.
- Altre chiavi opzionali come attention_mask.

In [ ]:
!python --version

Python 3.10.11


In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
     --- ------------------------------------ 0.5/6.1 MB 16.2 MB/s eta 0:00:01
     -------- ------------------------------- 1.2/6.1 MB 15.6 MB/s eta 0:00:01
     ------------ --------------------------- 1.9/6.1 MB 15.1 MB/s eta 0:00:01
     ----------------- ---------------------- 2.7/6.1 MB 15.8 MB/s eta 0:00:01
     ---------------------- ----------------- 3.4/6.1 MB 15.5 MB/s eta 0:00:01
     --------------------------- ------------ 4.1/6.1 MB 15.6 MB/s eta 0:00:01
     ------------------------------- -------- 4.8/6.1 MB 15.5 MB/s eta 0:00:01
     ------------------------------------- -- 5.7/6.1 MB 15.7 MB/s eta 0:00:01
     ---------------------------------------  6.1/6.1 MB 16.1 MB/s eta 0:00:01
     ---------------------------------------- 6.1/6.1 MB 14.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ----

ERROR: Could not install packages due to an OSError: [WinError 5] Accesso negato: 'C:\\Users\\verba\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\~orch\\lib\\asmjit.dll'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\verba\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------- ----------------- 1.4/2.4 GB 15.6 MB/s eta 0:01:09
     ---------------------- ----------------- 1.4/2.4 GB 15.2 MB/s eta 0:01:11
     ---------------------- ----------------- 1.4/2.4 GB 15.6 MB/s eta 0:01:09
     ---------------------- ----------------- 1.4/2.4 GB 14.9 MB/s eta 0:01:12
     ---------------------- ----------------- 1.4/2.4 GB 15.2 MB/s eta 0:01:11
     ---------------------- ----------------- 1.4/2.4 GB 14.9 MB/s eta 0:01:12
     ---------------------- ----------------- 1.4/2.4 GB 14.9 MB/s eta 0:01:12
     ---------------------- ----------------- 1.4/2.4 GB 14.6 MB/s eta 0:01:14
     ---------------------- ----------------- 1.4/2.4 GB 14.2 MB/s eta 0:01:15
     ---------------------- ----------------- 1.4/2.4 GB 14.2 MB/s eta 0:01:15
     ---------------------- ----------------- 1.4/2.4 GB 13.9 MB/s eta 0:01:17
     ---------------------- ----------------- 1.4/2.4 GB 13.9 MB/s eta 0:01:17
     ---------------------- ----------------- 1.4/2.

In [19]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Controlla se la GPU è supportata


2.5.1+cu121
False


In [ ]:
!nvidia-smi

Mon Dec  2 15:28:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              15W /  30W |   1029MiB /  8192MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Step 3: Scelta del Modello.
In questo passaggio, utilizzeremo il modello T5 (Text-to-Text Transfer Transformer), che è ampiamente utilizzato per compiti di abstractive summarization.

##Obiettivi dello Step 3
- Caricare un modello pre-addestrato T5.
- Configurare il modello per il fine-tuning sui dati preprocessati.
- Esegui il seguente codice per caricare il modello T5:

In [14]:
from transformers import AutoModelForSeq2SeqLM

# Carica il modello pre-addestrato T5 (versione piccola)
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Mostra un riassunto del modello
print(model.config)


T5Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stoppin

##Spiegazione del Codice
- AutoModelForSeq2SeqLM: Specifico per modelli che generano testo, come T5 o BART.
- t5-small: Una versione leggera del modello, adatta per iniziare e lavorare su risorse limitate. Puoi scegliere t5-base o t5-large per prestazioni migliori se hai capacità di calcolo sufficienti.

Optional: Verifica il Modello
Per assicurarti che il modello sia stato caricato correttamente, puoi testarlo su un esempio rapido:

In [17]:
# Test rapido con input di esempio
input_text = "summarize: " + "Though many attempts have been made in blind superresolution to restore low-resolution images with unknown and complex degradations, they are still far from addressing general real-world degraded images. In this work, we extend the powerful ESRGAN to a practical restoration application (namely, Real-ESRGAN), which is trained with pure synthetic data. Specifically, a high-order degradation modeling process is introduced to better simulate complex realworld degradations. We also consider the common ringing andovershoot artifacts in the synthesis process. In addition, we employ a U-Net discriminator with spectral normalization to increase discriminator capability and stabilize the training dynamics. Extensive comparisons have shown its superior visual performance than prior works on various real datasets. We also provide efficient implementations to synthesize training pairs on the fly."
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
output_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"Riassunto generato: {summary}")


Riassunto generato: a high-order degradation modeling process is introduced to better simulate complex realworld degradations. we also consider the common ringing andovershoot artifacts in the synthesis process.


#Step 4: Training o Fine-Tuning
In questo step, configureremo un trainer per affinare il modello T5 sui dati preprocessati del dataset CNN/DailyMail. Useremo la classe Seq2SeqTrainer di Hugging Face per semplificare il processo.

##Obiettivi dello Step 4
- Configurare i parametri del training (es. numero di epoche, batch size).
- Utilizzare Seq2SeqTrainer per il fine-tuning.
- Monitorare il processo di training.

##Spiegazione del Codice
- Seq2SeqTrainingArguments: Definisce i parametri del training, come tasso di apprendimento, dimensione del batch e numero di epoche.
- fp16: Usa calcoli in precisione ridotta per accelerare il training su GPU.
- Seq2SeqTrainer: Una classe specifica per il training di modelli di generazione sequenza-sequenza.
- evaluation_strategy: Valuta il modello periodicamente per monitorare i progressi.
##Monitoraggio
Durante il training, vedrai:

- Perdita (loss): Indica quanto il modello si sta adattando ai dati.
- Metriche di valutazione: Saranno calcolate se definite nel trainer.
##Note Importanti
- Risorse Hardware: Il training può essere intensivo. Se stai usando Google Colab, assicurati di attivare la GPU (Runtime > Change runtime type > GPU).
- Checkpoints: I modelli saranno salvati automaticamente nella cartella ./results.
##Prossimi Passi
Al termine del training, possiamo:

- Valutare le prestazioni del modello.
- Generare riassunti su dati di test.

##Obiettivo del Training
L'obiettivo di questo processo è migliorare il modello pre-addestrato T5 su un compito specifico: il riassunto astrattivo (abstractive summarization) di articoli di notizie. Sebbene il modello T5 sia già stato pre-addestrato su un vasto set di dati generici, il fine-tuning lo adatta ulteriormente per un'applicazione specifica (in questo caso, il dataset CNN/DailyMail).

In [ ]:
!pip install accelerate>=0.26.0


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\verba\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
#1. Installa o aggiorna accelerate
!pip install accelerate -U


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\verba\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
#assicurarti che tutte le dipendenze necessarie siano installate, usa il comando consigliato:
!pip install transformers[torch]
#2. Riavvia l'ambiente


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\verba\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
#3. Verifica l'installazione
import accelerate
print(accelerate.__version__)
#La versione dovrebbe essere almeno 0.26.0.

1.1.1


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Configurazione dei parametri del training
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",                 # Cartella per i risultati
    learning_rate=2e-5,                    # Tasso di apprendimento
    per_device_train_batch_size=8,         # Batch size per il training
    per_device_eval_batch_size=8,          # Batch size per la validazione
    weight_decay=0.01,                     # Decadimento del peso
    save_total_limit=3,                    # Limite per i checkpoint salvati
    num_train_epochs=3,                    # Numero di epoche
    predict_with_generate=True,            # Usa la generazione durante la valutazione
    logging_dir="./logs",                  # Cartella per i log
    logging_steps=100,                     # Frequenza dei log
    save_steps=500,                        # Salva il modello ogni 500 passi
    evaluation_strategy="steps",           # Valutazione ogni N passi
    eval_steps=500,                        # Passi per la valutazione
    fp16=True                              # Usa il calcolo in precisione ridotta (se disponibile)
)

# Configurazione del trainer
trainer = Seq2SeqTrainer(
    model=model,                           # Modello da affinare
    args=training_args,                    # Parametri di training
    train_dataset=processed_train_data,    # Dataset di training
    eval_dataset=processed_validation_data,# Dataset di validazione
    tokenizer=tokenizer                    # Tokenizer
)

# Avvia il training
trainer.train()

C:\Users\verba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\verba\AppData\Local\Temp\ipykernel_15224\2251491583.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
  0%|          | 101/107670 [00:17<4:26:37,  6.72it/s]

{'loss': 5.5481, 'grad_norm': 4.138925552368164, 'learning_rate': 1.9982167734745055e-05, 'epoch': 0.0}


  0%|          | 201/107670 [00:32<4:25:21,  6.75it/s]

{'loss': 1.4975, 'grad_norm': 1.8926321268081665, 'learning_rate': 1.996359245843782e-05, 'epoch': 0.01}


  0%|          | 301/107670 [00:47<4:24:20,  6.77it/s]

...

100%|█████████▉| 107601/107670 [10:10:02<00:10,  6.35it/s]  

{'loss': 0.9711, 'grad_norm': 0.7585477828979492, 'learning_rate': 2.0247051174886227e-08, 'epoch': 3.0}


100%|██████████| 107670/107670 [10:10:14<00:00,  2.94it/s]

{'train_runtime': 36614.4457, 'train_samples_per_second': 23.525, 'train_steps_per_second': 2.941, 'train_loss': 0.9968064139610525, 'epoch': 3.0}


TrainOutput(global_step=107670, training_loss=0.9968064139610525, metrics={'train_runtime': 36614.4457, 'train_samples_per_second': 23.525, 'train_steps_per_second': 2.941, 'total_flos': 1.16575171938091e+17, 'train_loss': 0.9968064139610525, 'epoch': 3.0})

# Step 5: Valutazione del Modello
In questa fase, misureremo l'efficacia del modello fine-tuned utilizzando metriche standard per il riassunto testuale, come ROUGE.

##Obiettivi dello Step 5
- Calcolare metriche di valutazione: ROUGE è la più utilizzata per i riassunti e misura l'overlap tra i riassunti generati e quelli di riferimento.
- Valutare su un sottoinsieme del dataset di test per evitare computazioni intensive.

In [11]:
!pip install -U datasets evaluate
!pip install nltk
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=171ba84cd08ccc2167df3e20b8fae57e199d0d688b43a0bfc1b8804e5b3e8d05
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [21]:
from evaluate import load
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Sposta il modello sul dispositivo

# Carica la metrica ROUGE
rouge = load("rouge")
print(next(model.parameters()).device)

# Funzione per generare riassunti e calcolare le metriche
def evaluate_model(model, tokenizer, dataset, num_samples=100):
    model.eval()  # Imposta il modello in modalità di valutazione
    model.to(device)  # Sposta il modello sul dispositivo

    results = []
    references = []

    for example in dataset.select(range(num_samples)):
        input_text = "summarize: " + example["article"]
        reference_summary = example["highlights"]

        # Sposta i dati sul dispositivo
        input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)

        # Genera il riassunto
        with torch.no_grad():  # Disabilita il calcolo del gradiente per risparmiare memoria
            summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        results.append(generated_summary)
        references.append(reference_summary)

    scores = rouge.compute(predictions=results, references=references)
    return scores


# Valutazione del modello sul dataset di test
scores = evaluate_model(model, tokenizer, dataset=test_data, num_samples=100)

# Stampa i risultati
print("Metriche ROUGE:")
print(scores)


cpu
Metriche ROUGE:
{'rouge1': 0.3184127043674146, 'rouge2': 0.13006430941006708, 'rougeL': 0.2400660625936858, 'rougeLsum': 0.2683501002486164}


#Valutazione delle Metriche
- ROUGE-1 (31.8%)
Misura l'overlap di parole singole (unigrammi) tra il riassunto generato e quello di riferimento. Un valore intorno al 30% è accettabile per compiti di riassunto astrattivo su dataset complessi come CNN/DailyMail, ma suggerisce che c'è spazio per miglioramenti.

- ROUGE-2 (13%)
Riguarda l'overlap di coppie di parole (bigrammi). Un valore del 13% è relativamente basso, il che può indicare che il modello ha difficoltà a catturare la coerenza locale o frasi significative.

- ROUGE-L (24%) e ROUGE-Lsum (26.8%)
Basano la valutazione sulla lunghezza della sottosequenza comune più lunga (longest common subsequence). Questi valori sono utili per capire quanto il modello riesce a catturare la struttura complessiva e la sintassi del riassunto di riferimento.

#Come si Collocano Questi Valori
- Per il dataset CNN/DailyMail, i modelli di base pre-addestrati come T5-small hanno metriche ROUGE-1 tra il 30-35% e ROUGE-2 tra il 10-15% senza un fine-tuning approfondito.
- Modelli avanzati e ben ottimizzati, come T5-base o BART, possono ottenere ROUGE-1 del 40-45% e ROUGE-2 del 20-25% dopo un fine-tuning esteso.
##Punti di Forza
- ROUGE-1 vicino al 32%: Dimostra che il modello cattura una buona quantità di parole chiave del riassunto di riferimento.
- Buona struttura globale: ROUGE-Lsum al 26.8% indica che il modello rispetta in parte la struttura del testo originale.
##Punti di Debolezza
- ROUGE-2 basso: Indica difficoltà nel generare frasi coese o nell'includere coppie di parole significative.
- Opportunità di miglioramento: L'addestramento aggiuntivo su dati più specifici o l'uso di tecniche avanzate (es. data augmentation o modelli più grandi) potrebbe migliorare i risultati.
##Suggerimenti per il Miglioramento
- Data Augmentation: Aumenta la varietà dei dati di training per aiutare il modello a generalizzare meglio.
- Hyperparameter Tuning: Ottimizza i parametri come il learning_rate o il numero di epoche.
- Modelli più avanzati: Usa versioni più grandi come T5-base o BART-large se hai risorse computazionali sufficienti.

###Spiegazione del Codice
####ROUGE:
1. ROUGE-1: Overlap di singole parole (unigrammi).
2. ROUGE-2: Overlap di coppie di parole (bigrammi).
3. ROUGE-L: Overlap basato sulla lunghezza della sequenza più lunga comune (longest common subsequence).

###Generazione Riassunto:
Usiamo il modello fine-tuned per generare riassunti.
La funzione generate include parametri come max_length e num_beams per controllare la lunghezza e la qualità dell'output.
###Valutazione:
La funzione rouge.compute confronta i riassunti generati con quelli di riferimento e calcola le metriche.

#Step 6: Inferenza
Qui useremo il modello T5 sia nella versione pre-addestrata che fine-tuned per generare e confrontare i riassunti di alcuni esempi di articoli.

##Obiettivi dello Step 6
- Generare riassunti con il modello pre-addestrato.
- Generare riassunti con il modello fine-tuned.
- Confrontare i riassunti generati con quelli di riferimento.


##Spiegazione del Codice
1. Funzione generate_summary:

  - Usa generate per creare riassunti.
  - Parametri chiave:
    - max_length: Lunghezza massima del riassunto generato.
    - num_beams: Numero di percorsi considerati durante la generazione (beam search).
  - Il prefisso "summarize:" indica il task al modello.
2. Confronto:

  - Usiamo due versioni del modello (pre-addestrato e fine-tuned) per generare riassunti dello stesso articolo.
  - Mostriamo i riassunti di riferimento dal dataset.
3. Input Limitato:

  - Mostriamo i primi 500 caratteri degli articoli per leggibilità.
##Risultati Attesi
- Modello Pre-addestrato: Riassunti più generici e meno specifici al dataset CNN/DailyMail.
- Modello Fine-Tuned: Riassunti più accurati e stilisticamente simili ai riassunti di riferimento.
- Confronto: Permette di evidenziare il miglioramento ottenuto con il fine-tuning.

In [22]:
from transformers import AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modello pre-addestrato
pre_trained_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Modello fine-tuned (carica il modello affinato)
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("./results/checkpoint-107670")  # Directory del modello salvato

def generate_summary(model, tokenizer, text, device, max_length=150, num_beams=4):
    model.eval()
    model.to(device)  # Sposta il modello sul dispositivo

    # Tokenizza il testo di input e sposta sul dispositivo
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", truncation=True, max_length=512).to(device)

    # Genera il riassunto
    with torch.no_grad():
        summary_ids = model.generate(
            input_ids,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True
        )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

i = randint(0, len(test_data)-1)
j = randint(0, len(test_data)-1)
# Esempio di articoli per l'inferenza
articles = [
    test_data[i]["article"],  # Primo articolo del dataset di test
    test_data[j]["article"],  # Secondo articolo
]

# Genera riassunti
for i, article in enumerate(articles):
    print(f"\n--- Articolo {i+1} ---")
    print(f"Testo originale:\n{article[:500]}...\n")  # Mostra i primi 500 caratteri per brevità

    # Confronto tra i due modelli
    print("--- Riassunto Modello Pre-addestrato ---")
    pre_trained_summary = generate_summary(pre_trained_model, tokenizer, article, device)
    print(pre_trained_summary)

    print("\n--- Riassunto Modello Fine-Tuned ---")
    fine_tuned_summary = generate_summary(fine_tuned_model, tokenizer, article, device)
    print(fine_tuned_summary)

    print("\n--- Riassunto di Riferimento ---")
    print(test_data[i]["highlights"])



--- Articolo 1 ---
Testo originale:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, includin...

--- Riassunto Modello Pre-addestrato ---
the palestinians signed the ICC's founding Rome Statute in January. the ICC also accepted its jurisdiction over alleged crimes committed in the occupied territories. the ICC opened a preliminary examination into the situation in Palestinian territories.

--- Riassunto Modello Fine-Tuned ---
The Palestinian Authority officially becomes the 123rd member of the International Criminal Court. The ICC opened a prelimina